In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer
from pydub import AudioSegment
from multiprocessing import Pool
import os

C:\Users\Komal Patil\AppData\Roaming\Python\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
data_url="https://asr.iitm.ac.in/Gramvaani/NEW/GV_Eval_3h.tar.gz"
data_path = keras.utils.get_file("GV_Eval_3h", data_url, untar=True)

In [3]:
wavs_path = data_path + "/Audio/"
matadata_path = data_path + "/text"

In [4]:
metadata_df = pd.read_csv(matadata_path, sep="|", header=None, quoting=3)

In [5]:
metadata_df.head(10)

,0
0,01-00004-02 वहीं राहुल कुमार मुकेश कुमार अरुण ...
1,01-00008-03 युवा की अहम् जिम्मेदारी है इसलिए ब...
2,01-00031-03 क्यूँकि हमारी पुलिस तो फेसबुक और द...
3,01-00071-02 नाली जाम है जिससे निकालता गन्दा पा...
4,01-00078-01 समस्या को सुना जायेगा और महिलाओं क...
5,01-00093-01 विकास के लिए शराब बंदी एक बड़ा कदम ...
6,01-00097-03 इस मध्य प्रदेश मोबाइल वाणी के सभी ...
7,01-00119-02 स्तर गिरने का मुख्य कारण संस्कृति...
8,01-00121-02 पुरुष के प्रतिनिधित्व की पूर्ति कर...
9,01-00129-02 जबकि प्रखंड कार्यालय के पीछे में थ...


In [6]:
split_data_df = metadata_df[0].str.split(' ', expand=True)

In [7]:
split_data_df.iloc[:, 1:101].fillna('')

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,वहीं,राहुल,कुमार,मुकेश,कुमार,अरुण,कुमार,राजा,कुमार,अनुराग,...,,,,,,,,,,
1,युवा,की,अहम्,जिम्मेदारी,है,इसलिए,बहुत,समय,या,पैसे,...,,,,,,,,,,
2,क्यूँकि,हमारी,पुलिस,तो,फेसबुक,और,दुसरे,सोशल,मीडिया,के,...,,,,,,,,,,
3,नाली,जाम,है,जिससे,निकालता,गन्दा,पानी,सड़को,पर,जमा,...,,,,,,,,,,
4,समस्या,को,सुना,जायेगा,और,महिलाओं,की,समस्या,महिलाओं,समझ,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,नमस्कार,श्रोता,मैं,मोबाइल,मिडिया,रिपोर्टर,खुर्शीद,आलम,बाईस,दिसम्बर,...,,,,,,,,,,
1028,की,सरकारी,कार्यालयों,में,बढ़ती,भ्रष्टाचार,का,मूल,कारण,सरकारी,...,,,,,,,,,,
1029,नमस्कार,श्रोताओ,में,मैं,धनबाद,मोबाइल,मीडिया,से,रिपोर्टर,खुर्शीद,...,,,,,,,,,,
1030,इसी,के,साथ,बताना,चाहता,हूँ,कि,कोई,ध्यान,नहीं,...,,,,,,,,,,


In [8]:
split_data_df['Merged_Column'] = split_data_df.iloc[:, 1:101].astype(str).apply(' '.join, axis=1)

In [9]:
metaadata_df = split_data_df.drop(split_data_df.iloc[:, 1:101], axis=1)

In [10]:
metaadata_df.head(10)

,0,Merged_Column
0,01-00004-02,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा क...
1,01-00008-03,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या प...
2,01-00031-03,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मी...
3,01-00071-02,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ज...
4,01-00078-01,समस्या को सुना जायेगा और महिलाओं की समस्या महि...
5,01-00093-01,विकास के लिए शराब बंदी एक बड़ा कदम है इस मुद्दे...
6,01-00097-03,इस मध्य प्रदेश मोबाइल वाणी के सभी सुनने वाले श...
7,01-00119-02,स्तर गिरने का मुख्य कारण संस्कृति और परम्पराओ...
8,01-00121-02,पुरुष के प्रतिनिधित्व की पूर्ति करते हैं और मह...
9,01-00129-02,जबकि प्रखंड कार्यालय के पीछे में थाना का जमीन ...


In [11]:
metaadata_df.columns = ["file_name", "translation"]
metaadata_df = metaadata_df[["file_name", "translation"]]
metaadata_df = metaadata_df.sample(frac=1).reset_index(drop=True)
metaadata_df.head(3)

,file_name,translation
0,01-06081-03,नमस्कार मुंगेर की आवाज में आपका स्वागत हैं मैं...
1,01-00264-02,मैं सभी को आग्रह करना चाहता हूँ ऐसा ही हमारी छ...
2,02-12495-01,उत्क्रमिक मध्य विद्यालय महादेव सिमरिया के पास ...


In [12]:
split = int(len(metaadata_df)* 0.90)
df_train = metaadata_df[:split]
df_val = metaadata_df[split:]

print(f"size of train set: {len(df_train)}")
print(f"size of train set: {len(df_val)}")

size of train set: 928
size of train set: 104


In [13]:
# Define the characters with unique entries
characters = list("अआइईउऊऋएऐओऔकखगघङचछजझञटठडढणतथदधनपफबभमयरलवशषसहक्षत्रज्ञ'?! ")

# Remove any duplicate entries
characters = list(set(characters))

# Create a StringLookup layer to map characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")

# Create a StringLookup layer to map integers back to characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

# Print the vocabulary
print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'ऐ', 'ए', 'ठ', 'ऋ', 'झ', 'च', 'थ', 'प', 'ण', 'ल', 'ग', 'छ', '!', 'भ', 'द', ' ', '?', 'ई', 'आ', 'ष', 'ढ', 'ऊ', 'न', 'औ', 'ड', 'क', 'त', 'श', '्', 'फ', 'ट', 'ङ', 'ब', 'स', 'घ', 'ख', 'ञ', 'इ', 'ज', 'अ', 'ध', 'उ', 'य', 'ह', 'र', "'", 'व', 'ओ', 'म'] (size=50)


In [14]:
frame_length = 256
frame_step = 160
fft_length = 384

def encode_single_sample(wav_file, label):  # Corrected typo in parameter name
    file = tf.io.read_file(wavs_path + wav_file + ".wav")
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    audio = tf.cast(audio, tf.float32)
    spectrogram = tf.signal.stft(
        audio, frame_length, frame_step=frame_step, fft_length=fft_length
    )
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)
    
    label = tf.strings.lower(label)
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    
    label = char_to_num(label)
    
    return spectrogram, label

In [15]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_train["file_name"]), list(df_train["translation"]))
)
train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_val["file_name"]), list(df_val["translation"]))
)
validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [16]:
fig = plt.figure(figsize=(8, 5))
for batch in train_dataset.take(1):
    spectrogram = batcch[0][0].numpy()
    spectrogram = np.array([np.trim_zeros(x) for x in np.transpose(spectrogram)])
    label = batch[1][0]
    
    label = tf.strings.reduce_join(num_to_char(label)).numpy.decode("utf-8")
    ax = plt.subplot(2, 1, 1)
    ax.imshow(spectrogram, vmax=1)
    ax.set_title(label)
    ax.axis("off")
    
    file = tf.io.read_file(wavs_path + list(df_train["file_name"])[0] + ".mp3")
    audio, _ =tf.audio.decode_mp3(file)
    audio = audio.numpy()
    ax = plt.subplot(2, 1, 2)
    plt.plot(audio)
    ax.set_title("Signal Wave")
    ax.set_xlim(0, len(audio))
    display.display(display.Audio(np.transpose(audio), rate=16000))
plt.show()

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: C:\Users\Komal Patil\.keras\datasets\GV_Eval_3h/Audio/01-06081-03.wav : The system cannot find the file specified.
; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

<Figure size 800x500 with 0 Axes>

In [17]:
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")
    
    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    input_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    
    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


In [18]:
def build_model(input_dim, output_dim, rnn_layers=5, rnn_units=128):
    # Model's input
    input_spectrogram = layers.Input((None, input_dim), name="input")

    # Expand the dimension to use 2D CNN
    x = layers.Reshape((-1, input_dim, 1), name="expand_dim")(input_spectrogram)

    # Convolution layer 1
    x = layers.Conv2D(
        filters=32,
        kernel_size=[11, 41],
        strides=[1, 2],
        padding="same",
        use_bias=False,
        name="conv1"
    )(x)
    x = layers.BatchNormalization(name="conv1_bn")(x)
    x = layers.ReLU(name="conv1_relu")(x)

    # Convolution layer 2
    x = layers.Conv2D(
        filters=32,
        kernel_size=[11, 21],
        strides=[1, 2],
        padding="same",
        use_bias=False,
        name="conv2"
    )(x)
    x = layers.BatchNormalization(name="conv2_bn")(x)
    x = layers.ReLU(name="conv2_relu")(x)

    # Reshape the resulted volume to feed the RNNs layers
    x = layers.Reshape((-1, x.shape[-2] * x.shape[-1]))(x)

    # RNN Layers
    for i in range(1, rnn_layers + 1):
        x = layers.Bidirectional(
            layers.GRU(
                units=rnn_units,
                activation="tanh",
                recurrent_activation="sigmoid",
                use_bias=True,
                return_sequences=True,
                reset_after=True,
                name=f"gru_{i}"
            ),
            name=f"bidirectional_{i}",
            merge_mode="concat"
        )(x)
        if i < rnn_layers:
            x = layers.Dropout(rate=0.5)(x)

    # Classification Layer
    output = layers.Dense(units=output_dim, activation="softmax", name="output")(x)

    # Model
    model = keras.Model(inputs=input_spectrogram, outputs=output, name="DeepSpeech2")

    # Optimizer
    opt = keras.optimizers.Adam(learning_rate=1e-4)

    # Compile the model and return
    model.compile(optimizer=opt, loss="categorical_crossentropy")

    return model


model = build_model(
    input_dim=fft_length // 2 + 1,
    output_dim=char_to_num.vocabulary_size(),
    rnn_units=512,
)
model.summary(line_length=110)

Model: "DeepSpeech2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                   ┃ Output Shape                        ┃             Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                             │ (None, None, 193)                   │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ expand_dim (Reshape)                           │ (None, None, 193, 1)                │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ conv1 (Conv2D)                                 │ (None, None, 97, 32)                │              14,432 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ conv1_bn (BatchNormalization)                  │ (None, None, 97, 32)                │                 128 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ conv1_relu (ReLU)                              │ (None, None, 97, 32)                │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ conv2 (Conv2D)                                 │ (None, None, 49, 32)                │             236,544 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ conv2_bn (BatchNormalization)                  │ (None, None, 49, 32)                │                 128 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ conv2_relu (ReLU)                              │ (None, None, 49, 32)                │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ reshape (Reshape)                              │ (None, None, 1568)                  │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ bidirectional_1 (Bidirectional)                │ (None, None, 1024)                  │           6,395,904 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ dropout (Dropout)                              │ (None, None, 1024)                  │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ bidirectional_2 (Bidirectional)                │ (None, None, 1024)                  │           4,724,736 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ dropout_1 (Dropout)                            │ (None, None, 1024)                  │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ bidirectional_3 (Bidirectional)                │ (None, None, 1024)                  │           4,724,736 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ dropout_2 (Dropout)                            │ (None, None, 1024)                  │                   0 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ bidirectional_4 (Bidirectional)                │ (None, None, 1024)                  │           4,724,736 │
├────────────────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┤
│ dropout_3 (Dropout)                            │ (None, None, 1024)                  │                   0 │
├───

 Total params: 25,597,330 (97.65 MB)

 Trainable params: 25,597,202 (97.65 MB)

 Non-trainable params: 128 (512.00 B)

In [22]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_train["file_name"]), list(df_train["translation"]))
)
train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_val["file_name"]), list(df_val["translation"]))
)
validation_dataset = (
    validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [ ]:
!pip install pydub